In [67]:
# imports
import pandas as pd
import itertools
import re
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
df = pd.read_csv('../datasets/spotify_dataset.csv', 
                         skiprows=1,
                         names=['user_id', 'artistname', 'trackname', 'playlistname'],
                         on_bad_lines='skip')

df.head()

,user_id,artistname,trackname,playlistname
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,"(What's So Funny 'Bout) Peace, Love And Unders...",HARD ROCK 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,Tiffany Page,7 Years Too Late,HARD ROCK 2010
3,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010
4,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,Alison,HARD ROCK 2010


In [3]:
# keep only 5% of the data
spotify_df = df.sample(frac=0.01, random_state=1)
spotify_df.head()

,user_id,artistname,trackname,playlistname
4361268,a846230a656e4f5181357ba60c2ed74d,Quiet Riot,Cum On Feel The Noize,music
8872077,8b77e77b5699d948cb7bd415b55f94e9,Bunbury,Bujías Para El Dolor,NisuViajes
12406318,e0a70ea54d3d19aa0eb3888914b9e90f,Alan Jackson,Where I Come From,Country
10202558,551f7a4022e603e092d48fc816dce2d6,Fionn Regan,List Of Distractions,Indie Fux Influx..
11402409,014e695cc6df96011b90a5beb3206012,Ilaiyaraaja,Ilan Cholai Poothatho,SPB Mothership


In [59]:
artists_set = set(itertools.chain(*[re.split(' [&\/] ', str(artist)) for artist in spotify_df.artistname]))
len(artists_set)

31823

In [60]:
# construct artists data set, where each artist have a list with associated tracks
artists_dict =dict((k, set()) for k in artists_set)
for _, row in spotify_df.iterrows():
    for artist in re.split(' [&\/] ', str(row.artistname)):
        artists_dict[artist].add(row.trackname)

    

In [62]:
# construct data frame with song
songs_list = []
for _, row in spotify_df.iterrows():
    song_entry = []

    # first column - song name
    song_entry.append(row.trackname)
    # second column - artists
    song_entry.append(re.split(' [&\/] ', str(row.artistname)))

    songs_list.append(song_entry)

song_df = pd.DataFrame(np.array(songs_list), columns=['song_name', 'song_performers'])

C:\Users\lazar\AppData\Local\Temp/ipykernel_34880/30032528.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  song_df = pd.DataFrame(np.array(songs_list), columns=['song_name', 'song_performers'])


In [63]:
song_df.head()

,song_name,song_performers
0,Cum On Feel The Noize,[Quiet Riot]
1,Bujías Para El Dolor,[Bunbury]
2,Where I Come From,[Alan Jackson]
3,List Of Distractions,[Fionn Regan]
4,Ilan Cholai Poothatho,[Ilaiyaraaja]


In [71]:
#function to scrape lyrics from genius
def scrape_lyrics(artistname, songname):
    artistname2 = str(artistname.replace(' ','-')) if ' ' in artistname else str(artistname)
    songname2 = str(songname.replace(' ','-')) if ' ' in songname else str(songname)
    return 'https://genius.com/'+ artistname2 + '-' + songname2 + '-' + 'lyrics'
    # page = requests.get('https://genius.com/'+ artistname2 + '-' + songname2 + '-' + 'lyrics')
    # html = BeautifulSoup(page.text, 'html.parser')
    # lyrics1 = html.find("div", class_="lyrics")
    # lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-2 jgQsqn")
    # if lyrics1:
    #     lyrics = lyrics1.get_text()
    # elif lyrics2:
    #     lyrics = lyrics2.get_text()
    # elif lyrics1 == lyrics2 == None:
    #     lyrics = None
    # return lyrics


In [73]:
requests.get(scrape_lyrics("Quiet Riot", "Cum On Feel The Noize"))

<Response [200]>

In [70]:
print(l)

None
